In [3]:
import pandas as pd
import json
import requests

## Formatting output from "Session Buddy"

In [15]:
with open('data.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data["tabs"])

In [18]:
output_df = df[['title','url']]
output_df.insert( 2,'user', "dataradar")
output_df.insert( 3,'tag1',"")
output_df.insert( 4,'tag2',"")
output_df.insert( 5,'tag3',"")
output_df.insert( 6,'tag4',"")
output_df.insert( 7,'tag5',"")
output_df = output_df.drop_duplicates()

In [19]:
output_df.to_json("output.json", orient="records", force_ascii=False)

## Removing Duplicates in consolidated.json File

In [35]:
with open('consolidated.json') as f:
    string = ""
    for line in f:
        string += line
        
data = json.loads(string)
df = pd.DataFrame(data)
df = df.drop_duplicates()

In [36]:
api_route = "https://www.feedcrunch.io/api/1.0/authenticated/get/article/exists/"

apikey_local = "91ce86bd-ae46-4250-b599-727c1970b609"
apikey_fc    = "4377190f-24a5-4ef7-a761-a877bf6218e3"

api_url = api_route + apikey_fc + "/"

def check_link_exists(url, user):
    r = requests.get(api_url+'?link='+url+'&posting_user='+user)
    try:
        return r.json()["exists"]
    except:
        return False

In [37]:
lines_2_delete = []

for index in range(df.shape[0]):
    
    url = df.iloc[index]['url']
    user = df.iloc[index]['user']
    
    if check_link_exists(url, user):
        lines_2_delete.append(index)
        print 'Duplicate Found: ' + url + " && user: " + user

In [31]:
df.drop(df.index[lines_2_delete], inplace=True)

In [33]:
df.to_json("consolidated.json", orient="records", force_ascii=False)